In [1]:
import boto3
import botocore
import pandas as pd
import math
from io import StringIO
import psycopg2
import os 
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# !pip install psycopg2-binary

In [3]:
def M3ToBbl(M3):
    m3ToBbl = M3 * 6.289811
    return m3ToBbl

def BblToM3(Bbl):
    bblToM3 = Bbl / M3ToBbl(1)
    return bblToM3

def BpdtoM3PerHr(Bpd):
    bpdtoM3PerHr = BblToM3(Bpd) / 24
    return bpdtoM3PerHr

def ReynoldsNumber(InnerDiameterInMm, ViscosityInCst, FlowInM3PerHr, FlowInKbpd, FlowInBpd):
    ## Check Flows
    if (FlowInM3PerHr > 0.0) and (FlowInKbpd == 0) and (FlowInBpd == 0):
        temp = 0
        ## left blank
    elif FlowInM3PerHr == 0.0 and FlowInKbpd > 0 and FlowInBpd == 0:
        FlowInM3PerHr = BpdtoM3PerHr(FlowInBpd)
    else:
        FlowInM3PerHr = -999999
    
    ## Calc return value
    if FlowInM3PerHr == -999999:
        reynoldsNum = 0 ##TODO: error
    else:
        reynoldsNum = 353678 * FlowInM3PerHr  / (InnerDiameterInMm * ViscosityInCst)
        
    return reynoldsNum


In [4]:
##Friction Factor
def FrictionFactorColebrookWhite(ReynoldsNumber, InnerDiameterInMm, RoughnessInMeters):
    if ReynoldsNumber == 0:
        return 0
    
    if ReynoldsNumber > 2300:
        RelRough = RoughnessInMeters / (InnerDiameterInMm / 1000)
        
        X1 = RelRough * ReynoldsNumber * 0.123968186335418
        X2 = math.log(ReynoldsNumber) - 0.779397488455682
        f = X2 - 0.2
        E = (math.log(X1 + f) + f - X2) / (1 + X1 + f)
        f = f - (1 + X1 + f + 0.5 * E) * E * (X1 + f) / (1 + X1 + f + E * (1 + E / 3))
        E = (math.log(X1 + f) + f - X2) / (1 + X1 + f)
        f = f - (1 + X1 + f + 0.5 * E) * E * (X1 + f) / (1 + X1 + f + E * (1 + E / 3))
        f = 1.15129254649702 / f
        return f * f
    else:
        return 64 / ReynoldsNumber

In [5]:
def FrictionalPressureLossInKpaPerKilometerDarcyWeisbach(FrictionFactor, FlowInM3PerHour, InnerDiameterInMm, DensityInKgPerM3):
    return (DensityInKgPerM3 * FrictionFactor * math.pow(FlowInM3PerHour, 2) * math.pow(1000, 5) * math.pow(4, 2)) / (math.pow(InnerDiameterInMm, 5) * math.pow(3600, 2) * math.pow(math.pi, 2) * 2)

In [6]:
def CtoK(C):
        kelvin = C + 273.15
        return kelvin


def Viscosity_ASTM_D341(T_in_C, V1_in_cSt, T1_in_C, V2_in_cSt, T2_in_C):
    print("1.", CtoK(T1_in_C))
    print("2.", CtoK(T2_in_C))
    x = math.log10(CtoK(T1_in_C))
    print("x=", x)
    y = math.log10(CtoK(T2_in_C))
    print("y=", y)
    print("3.", V2_in_cSt + 0.7)
    print("4.", math.log10(V2_in_cSt + 0.7))
    m = math.log10(math.log10(V2_in_cSt + 0.7))
    print("m=", m)
    n = math.log10(math.log10(V1_in_cSt + 0.7))
    print("n=", n)
    print (x-y)
    
    b = (math.log10(math.log10(V2_in_cSt + 0.7)) - math.log10(math.log10(V1_in_cSt + 0.7))) / (math.log10(CtoK(T1_in_C)) - math.log10(CtoK(T2_in_C)))
    
    ##if Vref_in_cSt != 0 and Tref_in_C != 0:
    ##    a = math.log(math.log(Vref_in_cSt + 0.7)) + (b * math.log(CtoK(Tref_in_C)))
    ##else:
    a = math.log10(math.log10(V1_in_cSt + 0.7)) + (b * math.log10(CtoK(T1_in_C)))
    
    viscosity = math.pow(10, (math.pow(10,(a - b * math.log10(CtoK(T_in_C)))))) - 0.7
    return viscosity

In [7]:
G = 9.80665
DISTANCEFROMSTATIONTOVALVE = 38.20408557 
STNELEVATION = 457.69              ##QUESTION: Does this change from station to station
VALVEELEVATION = 402.83            ##QUESTION: Does this change from station to station
ROUGHNESS = 0.0000457
DIN = 0.742

# viscosity5  = pd.read_csv('s3://commercial-liquids-dssanalytics-20190402/raw/Viscosity5/viscosityat5.csv', index_col=0, parse_dates=True)
# viscosity20  = pd.read_csv('s3://commercial-liquids-dssanalytics-20190402/raw/Viscosity20/viscosityat20.csv', index_col=0, parse_dates=True)

# dat  = pd.read_csv('s3://commercial-liquids-dssanalytics-20190402/raw/DSSTools/DSSTools.csv', index_col=0, parse_dates=True)
# linefill = pd.read_csv('s3://commercial-liquids-dssanalytics-20190402/raw/LineFill/Linefill.csv', index_col=12, parse_dates=True)


In [8]:
# read dataset from local driver c:\dev\Jupyter\data 
viscosity5  = pd.read_csv(r'data\viscosityat5.csv', index_col=0, parse_dates=True)
#
viscosity20  = pd.read_csv(r'data\viscosityat20.csv', index_col=0, parse_dates=True)

dra_df  = pd.read_csv(r'data\DSSTools.csv', index_col=0, parse_dates=True)
lf_df = pd.read_csv(r'data\Linefill.csv', index_col=12, parse_dates=True)


In [9]:
lf_df.index

DatetimeIndex(['2018-12-01 01:00:00', '2018-12-03 19:00:00',
               '2018-12-03 20:00:00', '2018-12-03 21:00:00',
               '2018-12-03 22:00:00', '2018-12-03 23:00:00',
               '2018-12-04 00:00:00', '2018-12-04 01:00:00',
               '2018-12-04 02:00:00', '2018-12-04 03:00:00',
               ...
               '2019-03-25 15:00:00', '2019-03-25 16:00:00',
               '2019-03-25 17:00:00', '2019-03-25 18:00:00',
               '2019-03-25 19:00:00', '2019-03-25 20:00:00',
               '2019-03-25 21:00:00', '2019-03-25 22:00:00',
               '2019-03-25 23:00:00', '2019-03-26 00:00:00'],
              dtype='datetime64[ns]', name='IntervalStartDateTime', length=2673, freq=None)

In [10]:
lf_transpose = lf_df.T

In [11]:
lf_transpose

IntervalStartDateTime,2018-12-01 01:00:00,2018-12-03 19:00:00,2018-12-03 20:00:00,2018-12-03 21:00:00,2018-12-03 22:00:00,2018-12-03 23:00:00,2018-12-04 00:00:00,2018-12-04 01:00:00,2018-12-04 02:00:00,2018-12-04 03:00:00,...,2019-03-25 15:00:00,2019-03-25 16:00:00,2019-03-25 17:00:00,2019-03-25 18:00:00,2019-03-25 19:00:00,2019-03-25 20:00:00,2019-03-25 21:00:00,2019-03-25 22:00:00,2019-03-25 23:00:00,2019-03-26 00:00:00
Segment,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,...,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone
BatchName,PDH/0023/0,WDB/0085/0,WDB/0085/0,WDB/0085/0,WDB/0085/0,CDB/5055/0,CDB/5055/0,CDB/5055/0,CDB/5055/0,CL/5704/0,...,FRB/0588/0,FRB/0588/0,KH/5039/0,KH/5039/0,KH/5039/0,KH/5039/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0
FacilityName,Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),...,Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future)
FacilityMnemonic,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,...,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN
BatchVolumeInPipeline,16131.9,15802.3,15802.3,15802.3,15802.3,16048.7,16048.7,16048.7,16048.7,23099.5,...,21926.5,21926.6,12358.1,12358.2,12358.2,3193.68,17760.9,17760.9,17761,17761
BatchVolumeInSegment,5474,4358,8065,11833,15694,3732,7868,12111,15365,2844,...,14186,18289,447,4428,8733,367,1407,5582,9782,13853
BatchVolume,15931,15899,15899,15899,15899,15899,15899,15899,15899,15971,...,18512,18512,15405,15405,15405,15405,17612,17612,17612,17612
Shipper,HollyFrontier,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Phillips 66 Co,Husky Mktg & Supply,...,Phillips 66 Can,Phillips 66 Can,ExxonMobil Oil Corp,ExxonMobil Oil Corp,ExxonMobil Oil Corp,ExxonMobil Oil Corp,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops
ReceiptFacility,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Gibson Terminal,Gibson Terminal,Gibson Terminal,Gibson Terminal,Enbridge HCT,...,Gibson West,Gibson West,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal
DeliveryFacility,Plains Cushing,WRB Terminal,WRB Terminal,WRB Terminal,WRB Terminal,WRB Terminal,WRB Terminal,WRB Terminal,WRB Terminal,Plains Patoka,...,Plains Cushing,Plains Cushing,Enterprise Appelt Terminal,Enterprise Appelt Terminal,Enterprise Appelt Terminal,Enterprise Appelt Terminal,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing


In [12]:
lf_df['2018']

,Segment,BatchName,FacilityName,FacilityMnemonic,BatchVolumeInPipeline,BatchVolumeInSegment,BatchVolume,Shipper,ReceiptFacility,DeliveryFacility,ScadaCommodityCode,Commodity,IntervalEndDateTime
IntervalStartDateTime,,,,,,,,,,,,,
2018-12-01 01:00:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00
2018-12-03 19:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00
2018-12-03 20:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00
2018-12-03 21:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,11833,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 22:00
2018-12-03 22:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,15694,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 23:00
2018-12-03 23:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,3732,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 0:00
2018-12-04 00:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,7868,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 1:00
2018-12-04 01:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,12111,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 2:00
2018-12-04 02:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,15365,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 3:00


In [18]:
dra_df.shape

(11041, 9)

In [19]:
dra_df.index

DatetimeIndex(['2018-12-01 00:00:00', '2018-12-01 00:15:00',
               '2018-12-01 00:30:00', '2018-12-01 00:45:00',
               '2018-12-01 01:00:00', '2018-12-01 01:15:00',
               '2018-12-01 01:30:00', '2018-12-01 01:45:00',
               '2018-12-01 02:00:00', '2018-12-01 02:15:00',
               ...
               '2019-03-25 21:45:00', '2019-03-25 22:00:00',
               '2019-03-25 22:15:00', '2019-03-25 22:30:00',
               '2019-03-25 22:45:00', '2019-03-25 23:00:00',
               '2019-03-25 23:15:00', '2019-03-25 23:30:00',
               '2019-03-25 23:45:00', '2019-03-26 00:00:00'],
              dtype='datetime64[ns]', name='SampleDateTime', length=11041, freq=None)

In [20]:
dra_df.isnull().sum()

Location_Ultrasonic_Flow             0
Location_Density                     0
Location_Discharge_Pressure          0
Downstream_Valve_Suction_Pressure    0
Location_Discharge_Temperature       0
Downstream_Suction_Temperature       0
DRA1_Concentration                   0
DRA1_Flow_Rate                       0
IntervalStartDateTime                0
dtype: int64

In [22]:
for j in range(0, len(lf_df) - 1):
    lf_df.at[lf_df.index[j],'NextBatchName'] = lf_df.loc[lf_df.index[j + 1], 'BatchName']
    print (j, lf_df.loc[lf_df.index[j],['BatchName','NextBatchName']])


0 BatchName        PDH/0023/0
NextBatchName    WDB/0085/0
Name: 2018-12-01 01:00:00, dtype: object
1 BatchName        WDB/0085/0
NextBatchName    WDB/0085/0
Name: 2018-12-03 19:00:00, dtype: object
2 BatchName        WDB/0085/0
NextBatchName    WDB/0085/0
Name: 2018-12-03 20:00:00, dtype: object
3 BatchName        WDB/0085/0
NextBatchName    WDB/0085/0
Name: 2018-12-03 21:00:00, dtype: object
4 BatchName        WDB/0085/0
NextBatchName    CDB/5055/0
Name: 2018-12-03 22:00:00, dtype: object
5 BatchName        CDB/5055/0
NextBatchName    CDB/5055/0
Name: 2018-12-03 23:00:00, dtype: object
6 BatchName        CDB/5055/0
NextBatchName    CDB/5055/0
Name: 2018-12-04 00:00:00, dtype: object
7 BatchName        CDB/5055/0
NextBatchName    CDB/5055/0
Name: 2018-12-04 01:00:00, dtype: object
8 BatchName        CDB/5055/0
NextBatchName     CL/5704/0
Name: 2018-12-04 02:00:00, dtype: object
9 BatchName        CL/5704/0
NextBatchName    CL/5704/0
Name: 2018-12-04 03:00:00, dtype: object
10 BatchName

95 BatchName        FRB/0327/0
NextBatchName     KH/1052/0
Name: 2018-12-07 17:00:00, dtype: object
96 BatchName        KH/1052/0
NextBatchName    KH/1052/0
Name: 2018-12-07 18:00:00, dtype: object
97 BatchName        KH/1052/0
NextBatchName    KH/1052/0
Name: 2018-12-07 19:00:00, dtype: object
98 BatchName         KH/1052/0
NextBatchName    FRB/0332/0
Name: 2018-12-07 20:00:00, dtype: object
99 BatchName        FRB/0332/0
NextBatchName    FRB/0332/0
Name: 2018-12-07 21:00:00, dtype: object
100 BatchName        FRB/0332/0
NextBatchName    FRB/0332/0
Name: 2018-12-07 22:00:00, dtype: object
101 BatchName        FRB/0332/0
NextBatchName    FRB/0332/0
Name: 2018-12-07 23:00:00, dtype: object
102 BatchName        FRB/0332/0
NextBatchName     CL/5726/0
Name: 2018-12-08 00:00:00, dtype: object
103 BatchName        CL/5726/0
NextBatchName    CL/5726/0
Name: 2018-12-08 01:00:00, dtype: object
104 BatchName        CL/5726/0
NextBatchName    CL/5726/0
Name: 2018-12-08 02:00:00, dtype: object
105

183 BatchName        LLB/0815/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 09:00:00, dtype: object
184 BatchName        CDB/5068/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 10:00:00, dtype: object
185 BatchName        CDB/5068/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 11:00:00, dtype: object
186 BatchName        CDB/5068/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 12:00:00, dtype: object
187 BatchName        CDB/5068/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 13:00:00, dtype: object
188 BatchName        CDB/5068/0
NextBatchName    CDB/5068/0
Name: 2018-12-11 14:00:00, dtype: object
189 BatchName        CDB/5068/0
NextBatchName    BHB/0729/0
Name: 2018-12-11 15:00:00, dtype: object
190 BatchName        BHB/0729/0
NextBatchName    BHB/0729/0
Name: 2018-12-11 16:00:00, dtype: object
191 BatchName        BHB/0729/0
NextBatchName    BHB/0729/0
Name: 2018-12-11 17:00:00, dtype: object
192 BatchName        BHB/0729/0
NextBatchName    BHB/0729/0
Name: 2018-12-11 18:00:00, dtyp

Name: 2018-12-15 22:00:00, dtype: object
272 BatchName        BHB/0726/0
NextBatchName    BHB/0726/0
Name: 2018-12-15 23:00:00, dtype: object
273 BatchName        BHB/0726/0
NextBatchName    BHB/0726/0
Name: 2018-12-16 00:00:00, dtype: object
274 BatchName        BHB/0726/0
NextBatchName    BHB/0726/0
Name: 2018-12-16 01:00:00, dtype: object
275 BatchName        BHB/0726/0
NextBatchName    BHB/0726/0
Name: 2018-12-16 02:00:00, dtype: object
276 BatchName        BHB/0726/0
NextBatchName    WCS/4158/0
Name: 2018-12-16 03:00:00, dtype: object
277 BatchName        WCS/4158/0
NextBatchName    WCS/4158/0
Name: 2018-12-16 04:00:00, dtype: object
278 BatchName        WCS/4158/0
NextBatchName    WCS/4158/0
Name: 2018-12-16 05:00:00, dtype: object
279 BatchName        WCS/4158/0
NextBatchName    WCS/4158/0
Name: 2018-12-16 06:00:00, dtype: object
280 BatchName        WCS/4158/0
NextBatchName    SMA/0235/0
Name: 2018-12-16 07:00:00, dtype: object
281 BatchName        SMA/0235/0
NextBatchName    S

366 BatchName        FRB/0404/0
NextBatchName    FRB/0404/0
Name: 2018-12-19 21:00:00, dtype: object
367 BatchName        FRB/0404/0
NextBatchName    FRB/0404/0
Name: 2018-12-19 22:00:00, dtype: object
368 BatchName        FRB/0404/0
NextBatchName    SMA/0223/0
Name: 2018-12-19 23:00:00, dtype: object
369 BatchName        SMA/0223/0
NextBatchName    SMA/0223/0
Name: 2018-12-20 00:00:00, dtype: object
370 BatchName        SMA/0223/0
NextBatchName    SMA/0223/0
Name: 2018-12-20 01:00:00, dtype: object
371 BatchName        SMA/0223/0
NextBatchName    SMA/0223/0
Name: 2018-12-20 02:00:00, dtype: object
372 BatchName        SMA/0223/0
NextBatchName    WCS/4159/0
Name: 2018-12-20 03:00:00, dtype: object
373 BatchName        WCS/4159/0
NextBatchName    WCS/4159/0
Name: 2018-12-20 04:00:00, dtype: object
374 BatchName        WCS/4159/0
NextBatchName    WCS/4159/0
Name: 2018-12-20 05:00:00, dtype: object
375 BatchName        WCS/4159/0
NextBatchName    WCS/4159/0
Name: 2018-12-20 06:00:00, dtyp

...

2115 BatchName        FRB/0544/0
NextBatchName    FRB/0544/0
Name: 2019-03-02 18:00:00, dtype: object
2116 BatchName        FRB/0544/0
NextBatchName    FRB/0544/0
Name: 2019-03-02 19:00:00, dtype: object
2117 BatchName        FRB/0544/0
NextBatchName    FRB/0544/0
Name: 2019-03-02 20:00:00, dtype: object
2118 BatchName        FRB/0544/0
NextBatchName    FRB/0544/0
Name: 2019-03-02 21:00:00, dtype: object
2119 BatchName        FRB/0544/0
NextBatchName     KH/5026/0
Name: 2019-03-02 22:00:00, dtype: object
2120 BatchName        KH/5026/0
NextBatchName    KH/5026/0
Name: 2019-03-02 23:00:00, dtype: object
2121 BatchName        KH/5026/0
NextBatchName    KH/5026/0
Name: 2019-03-03 00:00:00, dtype: object
2122 BatchName        KH/5026/0
NextBatchName    KH/5026/0
Name: 2019-03-03 01:00:00, dtype: object
2123 BatchName        KH/5026/0
NextBatchName    KH/5026/0
Name: 2019-03-03 02:00:00, dtype: object
2124 BatchName        KH/5026/0
NextBatchName    KH/5026/0
Name: 2019-03-03 03:00:00, dtyp

2272 BatchName        CL/5909/0
NextBatchName    CL/5909/0
Name: 2019-03-09 07:00:00, dtype: object
2273 BatchName        CL/5909/0
NextBatchName    CL/5909/0
Name: 2019-03-09 08:00:00, dtype: object
2274 BatchName        CL/5909/0
NextBatchName    CL/5909/0
Name: 2019-03-09 09:00:00, dtype: object
2275 BatchName         CL/5909/0
NextBatchName    SMA/0319/0
Name: 2019-03-09 10:00:00, dtype: object
2276 BatchName        SMA/0319/0
NextBatchName    SMA/0319/0
Name: 2019-03-09 11:00:00, dtype: object
2277 BatchName        SMA/0319/0
NextBatchName    SMA/0319/0
Name: 2019-03-09 12:00:00, dtype: object
2278 BatchName        SMA/0319/0
NextBatchName    SMA/0319/0
Name: 2019-03-09 13:00:00, dtype: object
2279 BatchName        SMA/0319/0
NextBatchName    SMA/0330/0
Name: 2019-03-09 14:00:00, dtype: object
2280 BatchName        SMA/0330/0
NextBatchName    SMA/0330/0
Name: 2019-03-09 15:00:00, dtype: object
2281 BatchName        SMA/0330/0
NextBatchName    SMA/0330/0
Name: 2019-03-09 16:00:00, 

2401 BatchName        FRB/0578/0
NextBatchName     CL/5910/0
Name: 2019-03-14 17:00:00, dtype: object
2402 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 18:00:00, dtype: object
2403 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 19:00:00, dtype: object
2404 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 20:00:00, dtype: object
2405 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 21:00:00, dtype: object
2406 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 22:00:00, dtype: object
2407 BatchName        CL/5910/0
NextBatchName    CL/5910/0
Name: 2019-03-14 23:00:00, dtype: object
2408 BatchName         CL/5910/0
NextBatchName    WCS/4271/0
Name: 2019-03-15 00:00:00, dtype: object
2409 BatchName        WCS/4271/0
NextBatchName    WCS/4271/0
Name: 2019-03-15 01:00:00, dtype: object
2410 BatchName        WCS/4271/0
NextBatchName    WCS/4271/0
Name: 2019-03-15 02:00:00, dtype:

2548 BatchName        BHB/0747/0
NextBatchName    BHB/0747/0
Name: 2019-03-20 20:00:00, dtype: object
2549 BatchName        BHB/0747/0
NextBatchName    BHB/0747/0
Name: 2019-03-20 21:00:00, dtype: object
2550 BatchName        BHB/0747/0
NextBatchName    BHB/0747/0
Name: 2019-03-20 22:00:00, dtype: object
2551 BatchName        BHB/0747/0
NextBatchName    BHB/0747/0
Name: 2019-03-20 23:00:00, dtype: object
2552 BatchName        BHB/0747/0
NextBatchName     CL/5924/0
Name: 2019-03-21 00:00:00, dtype: object
2553 BatchName        CL/5924/0
NextBatchName    CL/5924/0
Name: 2019-03-21 01:00:00, dtype: object
2554 BatchName        CL/5924/0
NextBatchName    CL/5924/0
Name: 2019-03-21 02:00:00, dtype: object
2555 BatchName        CL/5924/0
NextBatchName    CL/5924/0
Name: 2019-03-21 03:00:00, dtype: object
2556 BatchName        CL/5924/0
NextBatchName    CL/5924/0
Name: 2019-03-21 04:00:00, dtype: object
2557 BatchName        CL/5924/0
NextBatchName    CL/5924/0
Name: 2019-03-21 05:00:00, dtyp

2656 BatchName        KH/5038/0
NextBatchName    KH/5038/0
Name: 2019-03-25 08:00:00, dtype: object
2657 BatchName        KH/5038/0
NextBatchName    KH/5038/0
Name: 2019-03-25 09:00:00, dtype: object
2658 BatchName        KH/5038/0
NextBatchName    KH/5038/0
Name: 2019-03-25 10:00:00, dtype: object
2659 BatchName         KH/5038/0
NextBatchName    FRB/0588/0
Name: 2019-03-25 11:00:00, dtype: object
2660 BatchName        FRB/0588/0
NextBatchName    FRB/0588/0
Name: 2019-03-25 12:00:00, dtype: object
2661 BatchName        FRB/0588/0
NextBatchName    FRB/0588/0
Name: 2019-03-25 13:00:00, dtype: object
2662 BatchName        FRB/0588/0
NextBatchName    FRB/0588/0
Name: 2019-03-25 14:00:00, dtype: object
2663 BatchName        FRB/0588/0
NextBatchName    FRB/0588/0
Name: 2019-03-25 15:00:00, dtype: object
2664 BatchName        FRB/0588/0
NextBatchName     KH/5039/0
Name: 2019-03-25 16:00:00, dtype: object
2665 BatchName        KH/5039/0
NextBatchName    KH/5039/0
Name: 2019-03-25 17:00:00, dt

In [23]:
lf_df[['BatchName','NextBatchName']]

,BatchName,NextBatchName
IntervalStartDateTime,,
2018-12-01 01:00:00,PDH/0023/0,WDB/0085/0
2018-12-03 19:00:00,WDB/0085/0,WDB/0085/0
2018-12-03 20:00:00,WDB/0085/0,WDB/0085/0
2018-12-03 21:00:00,WDB/0085/0,WDB/0085/0
2018-12-03 22:00:00,WDB/0085/0,CDB/5055/0
2018-12-03 23:00:00,CDB/5055/0,CDB/5055/0
2018-12-04 00:00:00,CDB/5055/0,CDB/5055/0
2018-12-04 01:00:00,CDB/5055/0,CDB/5055/0
2018-12-04 02:00:00,CDB/5055/0,CL/5704/0


In [24]:
lf_df.isnull().sum()

Segment                  0
BatchName                0
FacilityName             0
FacilityMnemonic         0
BatchVolumeInPipeline    0
BatchVolumeInSegment     0
BatchVolume              0
Shipper                  0
ReceiptFacility          0
DeliveryFacility         0
ScadaCommodityCode       0
Commodity                0
IntervalEndDateTime      0
NextBatchName            1
dtype: int64

In [25]:
lf_df

,Segment,BatchName,FacilityName,FacilityMnemonic,BatchVolumeInPipeline,BatchVolumeInSegment,BatchVolume,Shipper,ReceiptFacility,DeliveryFacility,ScadaCommodityCode,Commodity,IntervalEndDateTime,NextBatchName
IntervalStartDateTime,,,,,,,,,,,,,,
2018-12-01 01:00:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-03 19:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00,WDB/0085/0
2018-12-03 20:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00,WDB/0085/0
2018-12-03 21:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,11833,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 22:00,WDB/0085/0
2018-12-03 22:00:00,Keystone,WDB/0085/0,Crandall Pump Station - (Future),CRNDL-SYN,15802.33,15694,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 23:00,CDB/5055/0
2018-12-03 23:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,3732,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 0:00,CDB/5055/0
2018-12-04 00:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,7868,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 1:00,CDB/5055/0
2018-12-04 01:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,12111,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 2:00,CDB/5055/0
2018-12-04 02:00:00,Keystone,CDB/5055/0,Crandall Pump Station - (Future),CRNDL-SYN,16048.71,15365,15899.0,Phillips 66 Co,Gibson Terminal,WRB Terminal,CDB5,CDB,12/4/2018 3:00,CL/5704/0


In [29]:
padded_lf = lf_df.asfreq('15Min', method='pad')
padded_lf

,Segment,BatchName,FacilityName,FacilityMnemonic,BatchVolumeInPipeline,BatchVolumeInSegment,BatchVolume,Shipper,ReceiptFacility,DeliveryFacility,ScadaCommodityCode,Commodity,IntervalEndDateTime,NextBatchName
IntervalStartDateTime,,,,,,,,,,,,,,
2018-12-01 01:00:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 01:15:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 01:30:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 01:45:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 02:00:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 02:15:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 02:30:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 02:45:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-01 03:00:00,Keystone,PDH/0023/0,Crandall Pump Station - (Future),CRNDL-SYN,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0


In [30]:
joined_table_merge = pd.merge(dra_df, padded_lf, left_index=True, right_index=True)

In [31]:
# after pad the number of dataset from 2673 to 11037 due to 15M intervals
padded_lf.T

IntervalStartDateTime,2018-12-01 01:00:00,2018-12-01 01:15:00,2018-12-01 01:30:00,2018-12-01 01:45:00,2018-12-01 02:00:00,2018-12-01 02:15:00,2018-12-01 02:30:00,2018-12-01 02:45:00,2018-12-01 03:00:00,2018-12-01 03:15:00,...,2019-03-25 21:45:00,2019-03-25 22:00:00,2019-03-25 22:15:00,2019-03-25 22:30:00,2019-03-25 22:45:00,2019-03-25 23:00:00,2019-03-25 23:15:00,2019-03-25 23:30:00,2019-03-25 23:45:00,2019-03-26 00:00:00
Segment,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,...,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone
BatchName,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,PDH/0023/0,...,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0,WCS/4288/0
FacilityName,Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),...,Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future),Crandall Pump Station - (Future)
FacilityMnemonic,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,...,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN,CRNDL-SYN
BatchVolumeInPipeline,16131.9,16131.9,16131.9,16131.9,16131.9,16131.9,16131.9,16131.9,16131.9,16131.9,...,17760.9,17760.9,17760.9,17760.9,17760.9,17761,17761,17761,17761,17761
BatchVolumeInSegment,5474,5474,5474,5474,5474,5474,5474,5474,5474,5474,...,1407,5582,5582,5582,5582,9782,9782,9782,9782,13853
BatchVolume,15931,15931,15931,15931,15931,15931,15931,15931,15931,15931,...,17612,17612,17612,17612,17612,17612,17612,17612,17612,17612
Shipper,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,HollyFrontier,...,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops,Husky Oil Ops
ReceiptFacility,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,Enbridge HCT,...,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal,Husky Terminal
DeliveryFacility,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,...,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing,Plains Cushing


In [32]:
joined_table_merge.shape

(11037, 23)

In [33]:
joined_table_merge.T

,2018-12-01 01:00:00,2018-12-01 01:15:00,2018-12-01 01:30:00,2018-12-01 01:45:00,2018-12-01 02:00:00,2018-12-01 02:15:00,2018-12-01 02:30:00,2018-12-01 02:45:00,2018-12-01 03:00:00,2018-12-01 03:15:00,...,2019-03-25 21:45:00,2019-03-25 22:00:00,2019-03-25 22:15:00,2019-03-25 22:30:00,2019-03-25 22:45:00,2019-03-25 23:00:00,2019-03-25 23:15:00,2019-03-25 23:30:00,2019-03-25 23:45:00,2019-03-26 00:00:00
Location_Ultrasonic_Flow,-1,-1.06171,-1.30089,-1.55117,-1.80145,-1.98923,295.128,408.371,934.992,2668.76,...,4111.67,4128.91,4142.4,4136.54,4131.48,4142.2,4147.83,4146.24,4142.06,4140.93
Location_Density,911.523,911.717,911.936,912.144,912.33,912.485,907.986,906.214,905.936,905.666,...,908.695,910.179,911.624,912.281,912.4,912.4,912.4,912.4,912.4,912.4
Location_Discharge_Pressure,464.456,433.241,402.472,371.967,341.938,311.885,364.501,436.813,877.99,3376.32,...,3581.38,3581.2,3577.4,3549.57,3528.04,3520.06,3504.04,3482.01,3464.34,3462.54
Downstream_Valve_Suction_Pressure,337.349,306.157,275.321,244.909,214.344,184.377,205.172,278.027,576.595,2426.2,...,2008.98,1996.06,1981.84,1954.42,1932.37,1921.79,1906.37,1888.01,1875.5,1872.62
Location_Discharge_Temperature,23.5391,23.3574,23.1899,23.028,22.8661,22.7102,22.1452,26.0132,26.681,27.521,...,23,23,23,23,23,23,23,23,23,23
Downstream_Suction_Temperature,25.2972,25.1836,25.1586,25.1335,25.1085,25.0836,25.0586,25.0335,25.0109,25.1639,...,20.3,20.3,20.3001,20.3142,20.3393,20.3643,20.3893,20.4,20.4,20.4
DRA1_Concentration,0,0,0,0,0,0,0,0,0,0,...,18.9691,18.9691,18.9691,18.9691,18.9691,18.9691,18.9691,18.9691,18.9691,18.9691
DRA1_Flow_Rate,0,0,0,0,0,0,0,0,0,0,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
IntervalStartDateTime,12/1/2018 1:00,12/1/2018 1:00,12/1/2018 1:00,12/1/2018 1:00,12/1/2018 2:00,12/1/2018 2:00,12/1/2018 2:00,12/1/2018 2:00,12/1/2018 3:00,12/1/2018 3:00,...,3/25/2019 21:00,3/25/2019 22:00,3/25/2019 22:00,3/25/2019 22:00,3/25/2019 22:00,3/25/2019 23:00,3/25/2019 23:00,3/25/2019 23:00,3/25/2019 23:00,3/26/2019 0:00
Segment,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,...,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone,Keystone


In [34]:
joined_table_merge.isnull().sum()

Location_Ultrasonic_Flow             0
Location_Density                     0
Location_Discharge_Pressure          0
Downstream_Valve_Suction_Pressure    0
Location_Discharge_Temperature       0
Downstream_Suction_Temperature       0
DRA1_Concentration                   0
DRA1_Flow_Rate                       0
IntervalStartDateTime                0
Segment                              0
BatchName                            0
FacilityName                         0
FacilityMnemonic                     0
BatchVolumeInPipeline                0
BatchVolumeInSegment                 0
BatchVolume                          0
Shipper                              0
ReceiptFacility                      0
DeliveryFacility                     0
ScadaCommodityCode                   0
Commodity                            0
IntervalEndDateTime                  0
NextBatchName                        1
dtype: int64

In [35]:
 joined_table_merge.index.minute

Int64Index([ 0, 15, 30, 45,  0, 15, 30, 45,  0, 15,
            ...
            45,  0, 15, 30, 45,  0, 15, 30, 45,  0],
           dtype='int64', length=11037)

In [36]:
lf_df.BatchName.value_counts(dropna=False)

CL/5704/0     22
WCS/4233/0    17
LLB/0851/0    13
WDB/0123/0    12
CDB/5098/0    11
LLB/0822/0    10
WDB/0114/0    10
CL/5924/0     10
BHB/0742/0     9
FRB/0471/0     8
WCS/4231/0     8
PDH/0031/0     8
SWB/1298/0     8
CDB/5077/0     8
SMA/0272/0     8
WDB/0122/0     8
WDB/0118/0     8
FRB/0456/0     8
FRB/0578/0     7
SMA/0287/0     7
LLB/0847/0     7
WCS/4164/0     7
FRB/0451/0     7
FRB/0344/0     7
WDB/0119/0     7
KH/1096/0      7
WCS/4201/0     7
KH/5029/0      7
CL/5910/0      7
WCS/4284/0     7
              ..
WDB/0096/0     3
LLB/0839/0     3
WDB/0110/0     3
CL/5791/0      3
KH/5022/0      3
SWB/1290/0     3
WDB/0116/0     3
FRB/0443/0     3
SMA/0228/0     3
FRB/0392/0     3
KH/5018/0      3
SMA/0225/0     3
KH/1078/0      3
FRB/0407/0     3
KH/1080/0      3
SWB/1277/0     3
KH/1073/0      3
FRB/0388/0     3
WCS/4211/0     3
SMA/0232/0     3
SWB/1281/0     3
KH/1095/0      3
SMA/0235/0     3
KH/1093/0      3
KH/1052/0      3
CL/5845/0      2
PDH/0023/0     1
FRB/0308/0    

In [37]:
##Remove rows that have missing LineFill data
for x in range(0, len(joined_table_merge) - 1):
    if joined_table_merge.index[x].minute == 0:
        if lf_df.loc[lf_df.index == joined_table_merge.index[x], 'BatchName'].empty:
            print (joined_table_merge.index[x])
            for y in range(1, 4):
                if(x-y>0):
                    joined_table_merge.at[joined_table_merge.index[x-y], 'ScadaCommodityCode'] = 'zzz'
                if(x+y<len(joined_table_merge)):
                    joined_table_merge.at[joined_table_merge.index[x+y], 'ScadaCommodityCode'] = 'zzz'
                joined_table_merge.at[joined_table_merge.index[x], 'ScadaCommodityCode'] = 'zzz'
joined_table = joined_table_merge[joined_table_merge.ScadaCommodityCode != 'zzz']

2018-12-01 02:00:00
2018-12-01 03:00:00
2018-12-01 04:00:00
2018-12-01 05:00:00
2018-12-01 06:00:00
2018-12-01 07:00:00
2018-12-01 08:00:00
2018-12-01 09:00:00
2018-12-01 10:00:00
2018-12-01 11:00:00
2018-12-01 12:00:00
2018-12-01 13:00:00
2018-12-01 14:00:00
2018-12-01 15:00:00
2018-12-01 16:00:00
2018-12-01 17:00:00
2018-12-01 18:00:00
2018-12-01 19:00:00
2018-12-01 20:00:00
2018-12-01 21:00:00
2018-12-01 22:00:00
2018-12-01 23:00:00
2018-12-02 00:00:00
2018-12-02 01:00:00
2018-12-02 02:00:00
2018-12-02 03:00:00
2018-12-02 04:00:00
2018-12-02 05:00:00
2018-12-02 06:00:00
2018-12-02 07:00:00
2018-12-02 08:00:00
2018-12-02 09:00:00
2018-12-02 10:00:00
2018-12-02 11:00:00
2018-12-02 12:00:00
2018-12-02 13:00:00
2018-12-02 14:00:00
2018-12-02 15:00:00
2018-12-02 16:00:00
2018-12-02 17:00:00
2018-12-02 18:00:00
2018-12-02 19:00:00
2018-12-02 20:00:00
2018-12-02 21:00:00
2018-12-02 22:00:00
2018-12-02 23:00:00
2018-12-03 00:00:00
2018-12-03 01:00:00
2018-12-03 02:00:00
2018-12-03 03:00:00


In [38]:
joined_table

,Location_Ultrasonic_Flow,Location_Density,Location_Discharge_Pressure,Downstream_Valve_Suction_Pressure,Location_Discharge_Temperature,Downstream_Suction_Temperature,DRA1_Concentration,DRA1_Flow_Rate,IntervalStartDateTime,Segment,...,BatchVolumeInPipeline,BatchVolumeInSegment,BatchVolume,Shipper,ReceiptFacility,DeliveryFacility,ScadaCommodityCode,Commodity,IntervalEndDateTime,NextBatchName
2018-12-01 01:00:00,-1.000000,911.523405,464.456057,337.348611,23.539056,25.297227,0.00000,0.000000,12/1/2018 1:00,Keystone,...,16131.92,5474,15931.0,HollyFrontier,Enbridge HCT,Plains Cushing,PDH5,PDH,12/1/2018 2:00,WDB/0085/0
2018-12-03 19:00:00,3824.288889,907.418945,3988.808889,2385.587908,29.734850,27.700001,0.00000,0.000000,12/3/2018 19:00,Keystone,...,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00,WDB/0085/0
2018-12-03 19:15:00,3394.176111,906.830358,3475.018333,2152.232648,29.695306,27.700001,0.00000,0.000000,12/3/2018 19:00,Keystone,...,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00,WDB/0085/0
2018-12-03 19:30:00,3672.348333,906.769016,4742.103148,3241.505556,29.964500,27.700001,0.00000,0.000000,12/3/2018 19:00,Keystone,...,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00,WDB/0085/0
2018-12-03 19:45:00,3736.396667,906.743995,5192.215494,3651.494444,30.000000,27.700001,0.00000,0.000000,12/3/2018 19:00,Keystone,...,15802.33,4358,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 20:00,WDB/0085/0
2018-12-03 20:00:00,3739.306111,906.718973,5305.047290,3763.246311,30.000000,27.700001,0.00000,0.000000,12/3/2018 20:00,Keystone,...,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00,WDB/0085/0
2018-12-03 20:15:00,3736.750556,906.700844,5315.396846,3776.549585,30.000000,27.700001,0.00000,0.000000,12/3/2018 20:00,Keystone,...,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00,WDB/0085/0
2018-12-03 20:30:00,3730.472778,906.700012,5308.353083,3774.763548,29.993432,27.700001,0.00000,0.000000,12/3/2018 20:00,Keystone,...,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00,WDB/0085/0
2018-12-03 20:45:00,3719.812222,906.700012,5265.993954,3742.174111,29.969355,27.700001,0.00000,0.000000,12/3/2018 20:00,Keystone,...,15802.33,8065,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 21:00,WDB/0085/0
2018-12-03 21:00:00,3714.709444,906.700012,5238.744203,3718.714491,29.944327,27.699878,0.00000,0.000000,12/3/2018 21:00,Keystone,...,15802.33,11833,15899.0,Phillips 66 Co,Enbridge HCT,WRB Terminal,WDB5,WDB,12/3/2018 22:00,WDB/0085/0


In [40]:
joined_table.columns

Index(['Location_Ultrasonic_Flow', 'Location_Density',
       'Location_Discharge_Pressure', 'Downstream_Valve_Suction_Pressure',
       'Location_Discharge_Temperature', 'Downstream_Suction_Temperature',
       'DRA1_Concentration', 'DRA1_Flow_Rate', 'IntervalStartDateTime',
       'Segment', 'BatchName', 'FacilityName', 'FacilityMnemonic',
       'BatchVolumeInPipeline', 'BatchVolumeInSegment', 'BatchVolume',
       'Shipper', 'ReceiptFacility', 'DeliveryFacility', 'ScadaCommodityCode',
       'Commodity', 'IntervalEndDateTime', 'NextBatchName'],
      dtype='object')

In [41]:
joined_table['Actual_Pressure_Drop'] = joined_table['Location_Discharge_Pressure'] - joined_table['Downstream_Valve_Suction_Pressure'] 
joined_table['Average_Temperature'] =  (joined_table['Location_Discharge_Temperature'] + joined_table['Downstream_Suction_Temperature']) / 2 

joined_table['accumulated_volume'] = 0.0
joined_table['calc'] = 0.0
joined_table['reset'] = False
joined_table['lastSlice'] = False
## Do the first line. our loop skips the first line because there is no previous row to look up
viscosityValueAt5 = viscosity5.loc[joined_table.loc[joined_table.index[0], 'ScadaCommodityCode'], 'Value']
viscosityValueAt20 = viscosity20.loc[joined_table.loc[joined_table.index[0], 'ScadaCommodityCode'], 'Value']
print("Commodity=", joined_table.loc[joined_table.index[0], 'ScadaCommodityCode'])
joined_table.at[joined_table.index[0], 'Estimated_Viscosity'] = Viscosity_ASTM_D341(round(joined_table.loc[joined_table.index[0], 'Average_Temperature']), viscosityValueAt5, 5, viscosityValueAt20, 20)
joined_table.at[joined_table.index[0], 'accumulated_volume'] = joined_table.at[joined_table.index[0], 'BatchVolumeInSegment']
joined_table.at[joined_table.index[0], 'Reynolds_Number'] = ReynoldsNumber(DIN*1000, joined_table.at[joined_table.index[0], 'Estimated_Viscosity'], joined_table.loc[joined_table.index[0], 'Location_Ultrasonic_Flow'], 0, 0)
joined_table.at[joined_table.index[0], 'Friction_Factor'] = FrictionFactorColebrookWhite(joined_table.loc[joined_table.index[0], 'Reynolds_Number'], DIN*1000, ROUGHNESS)
joined_table.at[joined_table.index[0], 'Frictional_Pressure_Loss'] = FrictionalPressureLossInKpaPerKilometerDarcyWeisbach(joined_table.loc[joined_table.index[0], 'Friction_Factor'], joined_table.loc[joined_table.index[0], 'Location_Ultrasonic_Flow'], DIN*1000, joined_table.loc[joined_table.index[0], 'Location_Density'])
joined_table.at[joined_table.index[0], 'Frictional_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[0], 'Frictional_Pressure_Loss'] * DISTANCEFROMSTATIONTOVALVE
joined_table.at[joined_table.index[0], 'Elevation_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[0], 'Location_Density'] * G * (VALVEELEVATION - STNELEVATION)/1000
joined_table.at[joined_table.index[0], 'Theoretical_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[0], 'Frictional_Pressure_Loss_Over_Segment'] + joined_table.loc[joined_table.index[0], 'Elevation_Pressure_Loss_Over_Segment'] 
joined_table.at[joined_table.index[0], 'DRA_Effectiveness'] = (joined_table.loc[joined_table.index[0], 'Theoretical_Pressure_Loss_Over_Segment'] - joined_table.loc[joined_table.index[0], 'Actual_Pressure_Drop']) / joined_table.loc[joined_table.index[0], 'Theoretical_Pressure_Loss_Over_Segment'] * 100

Commodity= PDH5
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 188.08721559999998
4. 2.2743592772991126
m= 0.3568590706523934
n= 0.4357216128545857
-0.022810834243969236


In [ ]:
for i in range(1, len(joined_table)):
    ##if(joined_table.loc[joined_table.index[i], 'Theoretical_Pressure_Loss_Over_Segment']) == 0:
    ##    print(i)
    if joined_table.index[i].minute == 0: ##reset from actual linefill data
        joined_table.at[joined_table.index[i], 'accumulated_volume'] = joined_table.loc[joined_table.index[i], 'BatchVolumeInSegment']
        if joined_table.at[joined_table.index[i], 'BatchName'] != joined_table.at[joined_table.index[i-1], 'BatchName']:
            ##Thamar wants the effectiveness from the second to last slice copied into the last slice to take into account the batch changing
            ##would not give a pure measurement for the effectiveness
            joined_table.at[joined_table.index[i-1], 'DRA_Effectiveness'] = joined_table.at[joined_table.index[i - 2], 'DRA_Effectiveness']
            joined_table.at[joined_table.index[i-1], 'lastSlice'] = True
    ##set accumulated_volume
    elif joined_table.loc[joined_table.index[i-1], 'BatchVolumeInPipeline'] - joined_table.loc[joined_table.index[i-1], 'accumulated_volume'] > (joined_table.loc[joined_table.index[i-1], 'Location_Ultrasonic_Flow']/4):
        joined_table.at[joined_table.index[i], 'calc'] = (joined_table.loc[joined_table.index[i], 'Location_Ultrasonic_Flow']/4)
        joined_table.at[joined_table.index[i], 'accumulated_volume'] = joined_table.loc[joined_table.index[i-1], 'accumulated_volume'] +  joined_table.loc[joined_table.index[i], 'calc']
        joined_table.at[joined_table.index[i], 'ScadaCommodityCode'] = joined_table.loc[joined_table.index[i-1], 'ScadaCommodityCode']
        joined_table.at[joined_table.index[i], 'Commodity'] = joined_table.loc[joined_table.index[i-1], 'Commodity']
        joined_table.at[joined_table.index[i], 'BatchName'] = joined_table.loc[joined_table.index[i-1], 'BatchName']
    else: ##start of a new batch
        joined_table.at[joined_table.index[i], 'accumulated_volume'] = (joined_table.loc[joined_table.index[i], 'Location_Ultrasonic_Flow']/4)
        if i + 4 < len(joined_table):   
            joined_table.at[joined_table.index[i], 'ScadaCommodityCode'] = joined_table.loc[joined_table.index[i+4], 'ScadaCommodityCode']
            joined_table.at[joined_table.index[i], 'Commodity'] = joined_table.loc[joined_table.index[i+4], 'Commodity']
        joined_table.at[joined_table.index[i], 'BatchName'] = joined_table.loc[joined_table.index[i], 'NextBatchName']
        joined_table.at[joined_table.index[i], 'reset'] = True
        joined_table.at[joined_table.index[i-1], 'lastSlice'] = True
        ##Thamar wants the effectiveness from the second to last slice copied into the last slice to take into account the batch changing
        ##would not give a pure measurement for the effectiveness
        joined_table.at[joined_table.index[i-1], 'DRA_Effectiveness'] = joined_table.at[joined_table.index[i - 2], 'DRA_Effectiveness']
    viscosityValueAt5 = viscosity5.loc[joined_table.loc[joined_table.index[i], 'ScadaCommodityCode'], 'Value']
    viscosityValueAt20 = viscosity20.loc[joined_table.loc[joined_table.index[i], 'ScadaCommodityCode'], 'Value']
    joined_table.at[joined_table.index[i], 'Estimated_Viscosity'] = Viscosity_ASTM_D341(round(joined_table.loc[joined_table.index[i], 'Average_Temperature']), viscosityValueAt5, 5, viscosityValueAt20, 20)
    joined_table.at[joined_table.index[i], 'Reynolds_Number'] = ReynoldsNumber(DIN*1000, joined_table.loc[joined_table.index[i], 'Estimated_Viscosity'], joined_table.loc[joined_table.index[i], 'Location_Ultrasonic_Flow'], 0, 0)
    joined_table.at[joined_table.index[i], 'Friction_Factor'] = FrictionFactorColebrookWhite(joined_table.loc[joined_table.index[i], 'Reynolds_Number'], DIN*1000, ROUGHNESS)
    joined_table.at[joined_table.index[i], 'Frictional_Pressure_Loss'] = FrictionalPressureLossInKpaPerKilometerDarcyWeisbach(joined_table.loc[joined_table.index[i], 'Friction_Factor'], joined_table.loc[joined_table.index[i], 'Location_Ultrasonic_Flow'], DIN*1000, joined_table.loc[joined_table.index[i], 'Location_Density'])
    joined_table.at[joined_table.index[i], 'Frictional_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[i], 'Frictional_Pressure_Loss'] * DISTANCEFROMSTATIONTOVALVE
    joined_table.at[joined_table.index[i], 'Elevation_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[i], 'Location_Density'] * G * (VALVEELEVATION - STNELEVATION)/1000
    joined_table.at[joined_table.index[i], 'Theoretical_Pressure_Loss_Over_Segment'] = joined_table.loc[joined_table.index[i], 'Frictional_Pressure_Loss_Over_Segment'] + joined_table.loc[joined_table.index[i], 'Elevation_Pressure_Loss_Over_Segment'] 
    joined_table.at[joined_table.index[i], 'DRA_Effectiveness'] = (joined_table.loc[joined_table.index[i], 'Theoretical_Pressure_Loss_Over_Segment'] - joined_table.loc[joined_table.index[i], 'Actual_Pressure_Drop']) / joined_table.loc[joined_table.index[i], 'Theoretical_Pressure_Loss_Over_Segment'] * 100 if joined_table.loc[joined_table.index[i], 'Theoretical_Pressure_Loss_Over_Segment'] != 0 else 0
    
        
joined_table = joined_table.drop('DeliveryFacility', 1).drop('ReceiptFacility', 1).drop('Shipper', 1).drop('IntervalEndDateTime', 1)
joined_table = joined_table.drop('reset', 1).drop('Segment', 1).drop('FacilityName', 1).drop('NextBatchName', 1)

## Add Calculated columns 2018-01-01 00:00
joined_table['Date_Time'] = joined_table.index
joined_table['DRA1_type'] = 'Heavy'


1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 193.2516245
4. 2.286123153345449
m= 0.359099622112508
n= 0.4348875274346294
-0.022810834243969236
1. 278.15
2. 293.15
x=

4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.24173383177

y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 174.475251
4. 2.241733831774202
m= 0.3505840461474665
n= 0.42892107903906307
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.46708989852

2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.43697184868639843
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.43697184868639843
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.43697184868639843
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.43697184868639843
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.43697184868639843
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 180.50507399999998
4. 2.256489414436852
m= 0.35343330051406574
n= 0.436971848686

-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m= 0.34782427962103984
n= 0.42962838088388555
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m= 0.34782427962103984
n= 0.42962838088388555
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m= 0.34782427962103984
n= 0.42962838088388555
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m= 0.34782427962103984
n= 0.42962838088388555
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m= 0.34782427962103984
n= 0.42962838088388555
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 168.86268169999997
4. 2.2275336820014218
m=

...

4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.00802620000002
4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.00802620000002
4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.00802620000002
4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.00802620000002
4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.00802620000002
4. 2.1003982087949815
m= 0.32230163925185656
n= 0.40423082744689803
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847

3. 1.456329313
4. 0.1632595910810078
m= -0.7871212956197928
n= -0.527752061350202
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.456329313
4. 0.1632595910810078
m= -0.7871212956197928
n= -0.527752061350202
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.3513045629999998
4. 0.13075324326582746
m= -0.8835475298795553
n= -0.5973590836985557
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.3513045629999998
4. 0.13075324326582746
m= -0.8835475298795553
n= -0.5973590836985557
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.3513045629999998
4. 0.13075324326582746
m= -0.8835475298795553
n= -0.5973590836985557
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.3513045629999998
4. 0.13075324326582746
m= -0.8835475298795553
n= -0.5973590836985557
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.44427

x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 160.754632
4. 2.206163495581068
m= 0.34363769423180596
n= 0.4240956652940898
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.44427906427

1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 153.0833573
4. 2.184927978251152
m= 0.33943712589975195
n= 0.41879747666452966
-0.022810834243969236
1. 2

n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0.42462330737014986
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 171.4943788
4. 2.2342498894158114
m= 0.3491317451113407
n= 0

In [ ]:
joined_table.index[0]

In [98]:
joined_table.loc['2018-12-01 01:00:00']

Location_Ultrasonic_Flow                                   -1
Location_Density                                      911.523
Location_Discharge_Pressure                           464.456
Downstream_Valve_Suction_Pressure                     337.349
Location_Discharge_Temperature                        23.5391
Downstream_Suction_Temperature                        25.2972
DRA1_Concentration                                          0
DRA1_Flow_Rate                                              0
IntervalStartDateTime                          12/1/2018 1:00
BatchName                                          PDH/0023/0
FacilityMnemonic                                    CRNDL-SYN
BatchVolumeInPipeline                                 16131.9
BatchVolumeInSegment                                     5474
BatchVolume                                             15931
ScadaCommodityCode                                       PDH5
Commodity                                                 PDH
Actual_P

In [104]:
csv_buffer = StringIO()
joined_table.to_csv('data\joined.csv', index=True)

# Upload CSV to S3
#s3_key = 'output/joined.csv'
#s3_bucket = 'commercial-liquids-dssanalytics-20190402'
#s3_resource = boto3.resource('s3')
#s3_resource.Object(s3_bucket, s3_key).put(Body=csv_buffer.getvalue())

## Regression test viscosity calculation

Use 4 test DSS temmperatures that were inputs to the Excel viscosity spreadsheet calucation located in OilCals.xla
These 4 temperatures will be used to produce test results using the **Viscosity_ASTM_D341** python function in this workbook

Iterate through the commodities for the 4 test benchmark temperature values and output the results

import into excel and pivot the data to align with the Excel known value calucations.

In [105]:
benchmark_tC = [10,21.746411,35.402044,39.700001]
calculationData = []
# viscosity5 was read in above
commodities = viscosity5.index.values

for testC in benchmark_tC:
    for commoditySeasonCode in commodities.tolist():
        viscosityValueAt5 = viscosity5.loc[commoditySeasonCode, 'Value']
        viscosityValueAt20 = viscosity20.loc[commoditySeasonCode, 'Value']
        #print( testC, commodityCode,viscosityValueAt5,viscosityValueAt20)
        if (commoditySeasonCode ==  'SPC') or (commoditySeasonCode =='WCC') : calculatedViscostiy = 99999999
        else:    calculatedViscostiy = Viscosity_ASTM_D341(round(testC), viscosityValueAt5, 5, viscosityValueAt20, 20)
        calculationData.append([testC, commoditySeasonCode,viscosityValueAt5,viscosityValueAt20,calculatedViscostiy])
        
#print(calculationData)
labels = ['testC', 'commoditySeasonCode','viscosityValueAt5','viscosityValueAt20','calculatedViscostiy']
calcs = pd.DataFrame.from_records(calculationData, columns=labels)
calcs['season_code'] = calcs['commoditySeasonCode'].str.extract('(\d)', expand=True)
calcs['season_code'].fillna('')
calcs['commodity_code'] = calcs['commoditySeasonCode'].str.extract('([A-Z]\D{0,})', expand=True)


#calcs.to_csv("TestResults-Viscosity_ASTM_D341.csv")

calcs.head()

1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 142.4295364
4. 2.1536000606869963
m= 0.3331650547820084
n= 0.41605301800488065
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 190.97271519999995
4. 2.2809713228260793
m= 0.3581198252079593
n= 0.4370950318094931
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 270.3260825
4. 2.431887950764721
m= 0.3859435609437791
n= 0.46363305546529693
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 163.8793428
4. 2.214524213647557
m= 0.34528043323040963
n= 0.4257982202068014
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 233.8318279
4. 2.36890362462122
m= 0.37454739244644825
n= 0.4530702511168101
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 126.2125551
4. 2.1011025588656373
m= 0.32244725167408156
n= 0.4031631972266209
-0.022810834243969236
1.

x= 2.444279064276847
y= 2.467089898520816
3. 188.12827519999996
4. 2.274454073809177
m= 0.3568771719012479
n= 0.4383581949347188
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 172.0629945
4. 2.235687476878707
m= 0.34941109413695465
n= 0.43323030903428705
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 165.3407906
4. 2.218380009936222
m= 0.3460359431125807
n= 0.42499115654804853
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 228.16425019999997
4. 2.3582475982016406
m= 0.3725894007962022
n= 0.4515158267334322
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 341.8963753
4. 2.5338944965393466
m= 0.40378852825671135
n= 0.4828339428899204
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 188.08721559999998
4. 2.2743592772991126
m= 0.3568590706523934
n= 0.4357216128545857
-0.022810834243969236
1. 278.1

1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.2310985959999998
4. 0.0902928360215062
m= -1.044346705940668
n= 0.01595733654596594
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.2504317829999998
4. 0.09706000388356711
m= -1.0129596956437321
n= -0.7490930693238286
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.222110658
4. 0.08711053158817264
m= -1.0599293359896613
n= -0.7336334289682537
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 8.201135886
4. 0.9138740078541778
m= -0.039113674578394986
n= 0.13530816957103334
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 12.59465833
4. 1.1001863903805134
m= 0.04146626830034824
n= 0.18938000665509547
-0.022810834243969236
1. 278.15
2. 293.15
x= 2.444279064276847
y= 2.467089898520816
3. 1.207546737
4. 0.08190394872242769
m= -1.0866951596923167
n= -0.7154198326206954
-0

ValueError: math domain error